## Train, evaluate and test the YOLOv12 model on Mapillary Dataset

* [The  Street View Text Dataset](http://www.iapr-tc11.org/mediawiki/index.php?title=The_Street_View_Text_Dataset)

#### Environment Setup

In [1]:
# Main mport necessary libraries and modules
import os, sys, platform, subprocess, json, time, re
import torch
import torchvision
import tensorflow as tf
import ultralytics
from ultralytics.utils.checks import check_requirements

In [2]:
# ===============================
# Diagnóstico do Ambiente YOLO

print("=== Ambiente Python & Bibliotecas ===")
print("Python version:   ", sys.version)
print("PyTorch version:  ", torch.__version__)
print("Torchvision ver.: ", torchvision.__version__)
print("Ultralytics ver.: ", ultralytics.__version__)

print("\n=== CUDA / GPU ===")
print("CUDA disponível:  ", torch.cuda.is_available())
print("Torch CUDA ver.:  ", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU detectada:   ", torch.cuda.get_device_name(0))
    print("Memória total:   ", torch.cuda.get_device_properties(0).total_memory // (1024**3), "GB")

print("\n=== Torchvision Ops ===")
print("NMS backend:     ", torchvision.ops.nms.__module__)

# ===============================
# Teste prático do NMS em CUDA
# ===============================
if torch.cuda.is_available():
    from torchvision.ops import nms
    try:
        boxes  = torch.rand((1000, 4), device="cuda")
        scores = torch.rand(1000, device="cuda")
        keep   = nms(boxes, scores, 0.5)
        print(f"NMS OK, output size: {keep.shape}")
    except Exception as e:
        print("Erro ao rodar NMS em CUDA:", e)
else:
    print("CUDA não disponível, NMS não testado.")

=== Ambiente Python & Bibliotecas ===
Python version:    3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:38:46) [MSC v.1929 64 bit (AMD64)]
PyTorch version:   2.3.1+cu121
Torchvision ver.:  0.18.1+cu121
Ultralytics ver.:  8.3.203

=== CUDA / GPU ===
CUDA disponível:   True
Torch CUDA ver.:   12.1
GPU detectada:    NVIDIA GeForce RTX 4090
Memória total:    23 GB

=== Torchvision Ops ===
NMS backend:      torchvision.ops.boxes
NMS OK, output size: torch.Size([918])


In [ ]:
# Ultralytics sanity check
check_requirements()

In [2]:
# Check Yolo supported
from ultralytics import YOLO
print(YOLO.__doc__)

# carrega um modelo base
#model = YOLO("yolo12n.pt")   # ou 'yolo12s.pt', 'yolo12m.pt', etc.
#print("Modelo carregado:", model)


    YOLO (You Only Look Once) object detection model.

    This class provides a unified interface for YOLO models, automatically switching to specialized model types
    (YOLOWorld or YOLOE) based on the model filename. It supports various computer vision tasks including object
    detection, segmentation, classification, pose estimation, and oriented bounding box detection.

    Attributes:
        model: The loaded YOLO model instance.
        task: The task type (detect, segment, classify, pose, obb).
        overrides: Configuration overrides for the model.

    Methods:
        __init__: Initialize a YOLO model with automatic type detection.
        task_map: Map tasks to their corresponding model, trainer, validator, and predictor classes.

    Examples:
        Load a pretrained YOLOv11n detection model
        >>> model = YOLO("yolo11n.pt")

        Load a pretrained YOLO11n segmentation model
        >>> model = YOLO("yolo11n-seg.pt")

        Initialize from a YAML configur

##### GPU Diagnostic

In [3]:
# Set to use the GPU explicitly
if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Atual:", torch.cuda.current_device())
    print("Nome:", torch.cuda.get_device_name(0))
    try:
        print(subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout)
    except Exception as e:
        print("nvidia-smi não disponível:", e)
else:
    print("Rodando em CPU.")

CUDA device count: 1
Atual: 0
Nome: NVIDIA GeForce RTX 4090
Thu Sep 25 08:02:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  |   00000000:01:00.0  On |                  Off |
|  0%   34C    P8             12W /  450W |     491MiB /  24564MiB |      0%      Default |
|                                         |                        |            

In [5]:
# Get the number of CPU cores
num_workers = os.cpu_count()

print(f"Available CPU cores (workers): {num_workers}")

Available CPU cores (workers): 32


In [6]:
# Calculate the optimal number of workers
optimal_workers = max(1, num_workers // 2)  # Use at least 1 worker
print(f"Optimal number of workers: {optimal_workers}")

Optimal number of workers: 16


### Main Execution

In [4]:
import os, yaml, math, importlib
from datetime import datetime
# from tqdm import tqdm # scripts .py
from tqdm.notebook import tqdm # tqdm no notebook

In [5]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [6]:
# Get current working directory
os.getcwd()

'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms'

In [7]:
# Import function from the modules train_yolov12
from src.train_yolov12 import create_project_structure, create_data_yaml, train_yolov12, predict_yolov12, evaluate_yolov12, create_data_yaml_mixed

In [ ]:
# Import and Reload the modules to ensure any changes are reflected
from src.convert_SvtDataset import convert_to_yolo_format, split_dataset

### Prepare Datasets to YOLOv12

#### SVT Dataset to Yolov12
* Does not used in this notebook

In [ ]:
# Function to run a Python script with arguments and print its output in real-time
def run_script(script_name, args=[]):
    """
    Run a Python script with real-time output display, including tqdm progress bars in Jupyter.

    Args:
        script_name (str): The path to the Python script.
        args (list): Additional arguments to pass to the script.

    Returns:
        None
    """
    # The `-u' argument forces unbuffered output. This is useful for tqdm to work properly.
    command = [sys.executable, '-u', script_name] + args

    # Executes process with direct terminal output (stdout and stderr)
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Exibe a saída em tempo real
    for line in iter(process.stdout.readline, ''):
        print(line, end='')  # Prints without blank lines

    process.stdout.close()
    return_code = process.wait()

    if return_code != 0:
        print(f"\nScript '{script_name}' exited with code {return_code}")
    else:
        print(f"\nScript '{script_name}' executed successfully!")

In [ ]:
# Run the script to reestructure and convert the SVT dataset to YOLOv11
script_path = os.path.join('scr', 'convert_SvtDataset.py')
run_script(script_path)

##### Define paths to the Dataset

In [ ]:
# Check that files are organised correctly
train_img_dir = 'svt_dataset/svt2yolo_final/train/images'
val_img_dir = 'svt_dataset/svt2yolo_final/val/images'
test_img_dir = 'svt_dataset/svt2yolo_final/test/images'

In [ ]:
check_directories(train_img_dir, val_img_dir, test_img_dir)

In [ ]:
# Check if some files exist
print("Training images:", os.listdir(train_img_dir)[:5])
print("Validation images:", os.listdir(val_img_dir)[:5])
print("Testing images:", os.listdir(test_img_dir)[:5])

### Training the YOLOv12 model

In [ ]:
# Create the default directory ‘yolov12’

train_yolov12.create_project_structure()  # Cria o diretório padrão "yolov12"

---
#### Treinar apenas com Mapillary

In [8]:
# Configuração de caminhos (Mapillary)

DATA_MAP = "data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch"
TRAIN_MAP = f"{DATA_MAP}/train/images"
VAL_MAP   = f"{DATA_MAP}/valid/images"
TEST_MAP  = f"{DATA_MAP}/test/images"
YAML_MAP  = f"{DATA_MAP}/data_local.yaml"  # criar se nao existir

for p in [TRAIN_MAP, VAL_MAP, TEST_MAP]:
    if not os.path.isdir(p):
        raise FileNotFoundError(p)

# cria data.yaml se necessário
if not os.path.exists(YAML_MAP):
    create_data_yaml(TRAIN_MAP, VAL_MAP, TEST_MAP, YAML_MAP, class_names=('text',))
else:
    print("Usando YAML existente:", YAML_MAP)

Usando YAML existente: data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch/data_local.yaml


In [9]:
# Auto-ajuste de batch size por VRAM

def suggest_batch(imgsz=1280, base=8):
    """
    Sugere batch_size dinamicamente baseado na VRAM da GPU.
    Ajustado para placas topo de linha (ex: RTX 4090).
    """
    print("=== Ajuste automático de batch pelo VRAM ===")
    if not torch.cuda.is_available():
        print("Nenhuma GPU detectada → usando CPU.")
        return max(4, base//2)

    try:
        # consulta VRAM total da GPU 0 via nvidia-smi
        out = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.total", "--format=csv,noheader,nounits"],
            capture_output=True, text=True, check=True
        ).stdout.strip().splitlines()

        mem = int(re.findall(r'\d+', out[0])[0])  # MB
        print(f"GPU detectada: {torch.cuda.get_device_name(0)} com {mem/1024:.1f} GB de VRAM")

        # heurística mais agressiva para RTX 4090
        if mem >= 24000:   # >= 24 GB (ex: RTX 4090)
            batch = 48 if imgsz >= 1280 else 64
            print(f"VRAM >= 24 GB → batch sugerido {batch}")
        elif mem >= 16000: # >= 16 GB (ex: RTX 4080, A4000)
            batch = 32 if imgsz >= 1280 else 48
            print(f"VRAM >= 16 GB → batch sugerido {batch}")
        elif mem >= 12000: # >= 12 GB (ex: 3060 Ti, 2080 Ti)
            batch = 24 if imgsz >= 1280 else 32
            print(f"VRAM >= 12 GB → batch sugerido {batch}")
        elif mem >= 8000:  # >= 8 GB (ex: 3070, 4060 Ti)
            batch = 16 if imgsz >= 1280 else 24
            print(f"VRAM >= 8 GB → batch sugerido {batch}")
        else:
            batch = base
            print(f"VRAM baixa (<8 GB) → batch mantido {batch}")

        return batch

    except Exception as e:
        print(f"Erro ao consultar VRAM com nvidia-smi: {e}")
        print(f"Batch default {base}")
        return base

# ===== Uso =====
BATCH_MAP = suggest_batch(imgsz=1280, base=8)
print("Batch final sugerido:", BATCH_MAP)


=== Ajuste automático de batch pelo VRAM ===
GPU detectada: NVIDIA GeForce RTX 4090 com 24.0 GB de VRAM
VRAM >= 24 GB → batch sugerido 48
Batch final sugerido: 48


In [10]:
# Ativar otimização do PyTorch para acelerar treino no RTX 4090
torch.backends.cudnn.benchmark = True  
# o cuDNN escolher dinamicamente a melhor implementação para convoluções 
# dependendo do tamanho do batch/entrada.

torch.backends.cuda.matmul.allow_tf32 = True  
# Permite uso de TensorFloat32 (TF32) em multiplicações de matriz na GPU.
# O RTX 30xx/40xx tem núcleos otimizados para TF32 → muito mais rápido que FP32,
# com precisão suficiente para deep learning.

torch.set_float32_matmul_precision('high')  
# -> Define precisão para operações de matmul. Opções: "highest", "high", "medium".
#    "high" geralmente equilibra melhor velocidade e estabilidade numérica.

In [ ]:
# Teste rápido
model = YOLO("yolov12/weights/yolo12x.pt")
model.train(data=YAML_MAP, epochs=1, batch=4, imgsz=640, cache='disk', workers=2)

In [16]:
# Treinamento
WEIGHTS_INIT = "yolov12/weights/yolo12x.pt"
RUNS_DIR     = "yolov12/runs"
RUN_NAME     = "mapillary_only_v1"

model_map, results_map = train_yolov12(
    data_yaml=YAML_MAP,
    img_size=640,
    batch_size=8, #BATCH_MAP
    epochs=200,
    weights= WEIGHTS_INIT,
    device='0' if torch.cuda.is_available() else 'cpu',
    workers=4, #optimal_workers
    cache='disk',
    project=RUNS_DIR,
    name=RUN_NAME,
    #  Augmentations explícitas
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.0, fliplr=0.5,
    translate=0.08, scale=0.2, shear=0.0,
    perspective=0.0, mosaic=0.1, mixup=0.0, close_mosaic=10
)

[train] weights=yolov12/weights/yolo12x.pt | device=0 | imgsz=640 | batch=8
Ultralytics 8.3.203  Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch/data_local.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov12/weights/yolo12x.pt, momentum=0.9, mosaic=0.1, mul

##### Mapillary: Predição + avaliação no TEST

In [14]:
# === Predição ===
RUNS_DIR     = "yolov12/runs"
RUN_NAME     = "mapillary_only_v1"

BEST_MAP = os.path.join(RUNS_DIR, RUN_NAME, "weights", "best.pt")
assert os.path.exists(BEST_MAP), f"Pesos não encontrados: {BEST_MAP}"

model, results_pred = predict_yolov12(
    weights=BEST_MAP,
    source_images_dir=TEST_MAP,
    img_size=640,
    conf_thres=0.25, iou_thres=0.45,
    device='cuda:0',
    workers=4,
    save_txt=True, save_img=True, save_crop=True,
    project=RUNS_DIR,
    name=f"{RUN_NAME}_pred"
)

print("Predição concluída. Resultados salvos em:", os.path.join(RUNS_DIR, f"{RUN_NAME}_pred"))

[predict] weights=yolov12/runs\mapillary_only_v1\weights\best.pt | imgsz=640 | conf=0.25 | iou=0.45


Inferência:   0%|          | 0/1 [00:00<?, ?img/s]

Results saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mapillary_only_v1_pred
25 labels saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mapillary_only_v1_pred\labels
[predict] concluído → 26 resultados
Predição concluída. Resultados salvos em: yolov12/runs\mapillary_only_v1_pred


In [ ]:
# === Avaliação no TEST ===
RUNS_DIR     = "yolov12/runs"
RUN_NAME     = "mapillary_only_v1"

metrics = evaluate_yolov12(
    model=model,
    data_yaml=YAML_MAP,
    img_size=640,
    conf_thres=0.25,
    iou_thres=0.45,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    workers=4,
    cache="disk",
    project=RUNS_DIR,
    name=f"{RUN_NAME}_eval"
)
print("Métricas:", metrics.results_dict)

val: Fast image access  (ping: 0.00.0 ms, read: 941.6261.3 MB/s, size: 65.5 KB)
val: Scanning C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\test\labels.cache... 26 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 26/26  0.0s
WARNING Box and segment counts should be equal, but got len(segments) = 73, len(boxes) = 74. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
val: Caching images (0.0GB Disk): 100% ━━━━━━━━━━━━ 26/26 1.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.2it/s 13.1s5.6s
                   all         26         74      0.872      0.784      0.859      0.769
Speed: 0.2ms preprocess, 437.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\m

---
#### Treinamento conjunto (Mapillary + SVT)

##### Preparar dataset conjunto (Mapillary + SVT)

In [14]:
# Configuração de caminhos (Mapillary)
# Dataset misto (Mapillary + SVT)
DATA_MAP_MIX= "data\dataset_yolo"

DATA_SVT = "data\dataset_yolo\svt2yolo_final" 
TRAIN_SVT = f"{DATA_SVT}/train/images"
VAL_SVT   = f"{DATA_SVT}/val/images"
TEST_SVT  = f"{DATA_SVT}/test/images"

for p in [TRAIN_SVT, VAL_SVT, TEST_SVT]:
    if not os.path.isdir(p):
        raise FileNotFoundError(p)

YAML_MIX = os.path.join(DATA_MAP_MIX, "data_mixed.yaml")
create_data_yaml_mixed(
    train_list=[TRAIN_MAP, TRAIN_SVT],
    val_list=[VAL_MAP, VAL_SVT],
    test_list=[TEST_MAP, TEST_SVT],
    output_path=YAML_MIX,
    class_names=('text',)
)
print("Usando YAML existente:", YAML_MIX)

data.yaml (mixed): {'train': ['c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\\train\\images', 'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\svt2yolo_final\\train\\images'], 'val': ['c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\\valid\\images', 'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\svt2yolo_final\\val\\images'], 'test': ['c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\\test\\images', 'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\data\\dataset_yolo\\svt2yolo_final\\test\\images'], 'nc': 1, 'names': ['text']}
Usando YAML existente: data\dataset_yolo\data_mixed.yaml


##### Treinar conjunto (transferindo dos pesos do Mapillary)

In [ ]:
# Treinar misto (Mapillary + SVT) com transfer learning

RUNS_DIR     = "yolov12/runs"
RUN_NAME     = "mapillary_only_v1"
RUN_NAME_MIX     = "mixed_mapillary_svt_v1"

BEST_MAP = os.path.join(RUNS_DIR, RUN_NAME, "weights", "best.pt")

WEIGHTS_MIX_INIT = BEST_MAP    # transfer learning do melhor peso do Mapillary

#BATCH_MIX = suggest_batch(imgsz=640, base=8)
#print("Batch sugerido (Mixed):", BATCH_MIX)

model_mix, results_mix = train_yolov12(
    data_yaml=YAML_MIX,
    img_size=640,
    batch_size=16, # BATCH_MIX
    epochs=200,
    weights=WEIGHTS_MIX_INIT,
    device='0' if torch.cuda.is_available() else 'cpu',
    workers=4,
    cache='disk',
    project=RUNS_DIR,
    name=RUN_NAME_MIX,
    # Augmentations explícitas
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.0, fliplr=0.5,
    translate=0.08, scale=0.2, shear=0.0,
    perspective=0.0, mosaic=0.1, mixup=0.0, auto_augment='none', close_mosaic=10
)

=== Ajuste automático de batch pelo VRAM ===
GPU detectada: NVIDIA GeForce RTX 4090 com 24.0 GB de VRAM
VRAM >= 24 GB → batch sugerido 64
Batch sugerido (Mixed): 64
[train] weights=yolov12/runs\mapillary_only_v1\weights\best.pt | device=0 | imgsz=640 | batch=16
Ultralytics 8.3.203  Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=none, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\dataset_yolo\data_mixed.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00

##### Predição + avaliação (Mapillary + SVT)

In [18]:
# === Predição misto (Mapillary + SVT) ===
import yaml
from pathlib import Path

DATA_MAP_MIX= "data\dataset_yolo"
RUNS_DIR     = "yolov12/runs"
RUN_NAME_MIX     = "mixed_mapillary_svt_v1"

BEST_MIX = os.path.join(RUNS_DIR, RUN_NAME_MIX, "weights", "best.pt")
assert os.path.exists(BEST_MAP), f"Pesos não encontrados: {BEST_MAP}"

# Carrega YAML que contém os caminhos de teste
YAML_MIX = os.path.join(DATA_MAP_MIX, "data_mixed.yaml")
with open(YAML_MIX) as f:
    data_yaml = yaml.safe_load(f)

# Nomes manuais para subpastas dos resultados
subpastas = ['mpl_pred', 'svt_pred']
assert len(data_yaml['test']) == len(subpastas), "Número de caminhos de teste e subpastas não bate!"

# Loop nas duas pastas de teste
for test_path, nome_subpasta in zip(data_yaml['test'], subpastas):
    print(f"Rodando predição para: {test_path}")

    model, results_pred = predict_yolov12(
        weights=BEST_MIX,
        source_images_dir=test_path,
        img_size=640,
        conf_thres=0.25,
        iou_thres=0.45,
        device='cuda:0',
        workers=4,
        save_txt=True,
        save_img=True,
        save_crop=True,
        project=RUNS_DIR,
        name=f"{RUN_NAME_MIX}_pred/{nome_subpasta}"
    )
    print(f"Predição concluída para: {nome_subpasta}")

Rodando predição para: c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\test\images
[predict] weights=yolov12/runs\mixed_mapillary_svt_v1\weights\best.pt | imgsz=640 | conf=0.25 | iou=0.45


Inferência:   0%|          | 0/1 [00:00<?, ?img/s]

Results saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mixed_mapillary_svt_v1_pred\mpl_pred
26 labels saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mixed_mapillary_svt_v1_pred\mpl_pred\labels
[predict] concluído → 26 resultados
Predição concluída para: mpl_pred
Rodando predição para: c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\data\dataset_yolo\svt2yolo_final\test\images
[predict] weights=yolov12/runs\mixed_mapillary_svt_v1\weights\best.pt | imgsz=640 | conf=0.25 | iou=0.45


Inferência:   0%|          | 0/1 [00:00<?, ?img/s]

Results saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mixed_mapillary_svt_v1_pred\svt_pred
34 labels saved to C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\yolov12\runs\mixed_mapillary_svt_v1_pred\svt_pred\labels
[predict] concluído → 36 resultados
Predição concluída para: svt_pred


In [21]:
# === Avaliaçao misto (Mapillary + SVT) ===

# Diretórios
DATA_MAP_MIX= "data\dataset_yolo"
RUNS_DIR     = "yolov12/runs"
RUN_NAME_MIX     = "mixed_mapillary_svt_v1"
EVAL_DIR = f"{RUN_NAME_MIX}_eval"

# Carrega YAML que contém os caminhos de teste
YAML_MIX = os.path.join(DATA_MAP_MIX, "data_mixed.yaml")
with open(YAML_MIX) as f:
    data_yaml = yaml.safe_load(f)

# Subpastas para salvar resultados
subpastas = ['mpl_eval', 'svt_eval']
assert len(data_yaml['test']) == len(subpastas), "Número de conjuntos de teste não bate com subpastas."

# Avaliação separada para cada conjunto de teste
for test_path, nome_subpasta in zip(data_yaml['test'], subpastas):
    print(f"Avaliando conjunto: {test_path}")

    # Cria um YAML temporário apenas com esse conjunto
    temp_yaml = f"temp_data_eval_{nome_subpasta}.yaml"
    data_temp = {
        'train': "",         # ignorado na avaliação
        'val': "",           # ignorado também
        'test': test_path,
        'nc': data_yaml['nc'],
        'names': data_yaml['names']
    }
    with open(temp_yaml, 'w') as f:
        yaml.dump(data_temp, f)

    # Avaliação
    metrics = evaluate_yolov12(
        model=model_mix,
        data_yaml=temp_yaml,
        img_size=640,
        conf_thres=0.25,
        iou_thres=0.45,
        device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
        workers=4,
        cache="disk",
        project=RUNS_DIR,
        name=f"{EVAL_DIR}/{nome_subpasta}"
    )

    print(f"Avaliação concluída para: {nome_subpasta}")
    print("Métricas:", metrics.results_dict)

Avaliando conjunto: c:\DEV\PhD_Thesis_Step3_OSM_Toponyms\data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\test\images
val: Fast image access  (ping: 0.00.0 ms, read: 716.184.6 MB/s, size: 58.7 KB)
val: Scanning C:\DEV\PhD_Thesis_Step3_OSM_Toponyms\data\dataset_yolo\DSc-sli-mapillary-place-names-si.v1-v1.yolov5pytorch\test\labels.cache... 26 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 26/26 26.0Kit/s 0.0s
WARNING Box and segment counts should be equal, but got len(segments) = 73, len(boxes) = 74. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
val: Caching images (0.0GB Disk): 100% ━━━━━━━━━━━━ 26/26 13.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.8it/s 2.4s7.4s
                   all         26         74      0.892      0.811      0.867       0.75
Spee